# 1. 테스트 데이터 전처리

In [1]:
%pip install "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

  Using cached torch-2.4.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.19.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torchaudio-2.4.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (

In [2]:
!pip install vllm==0.8.2 datasets

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchaudio-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
 

In [3]:
!pip install rich


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import re
import json
import pandas as pd
from typing import List, Dict
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams

INFO 12-28 13:04:04 [__init__.py:239] Automatically detected platform cuda.


In [5]:
# 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("HJUNN/crypto_function_calling_datasets", split = "train")

In [6]:
# 테스트 비율 설정
test_ratio = 0.2

total_len = len(dataset)
test_size = int(total_len * test_ratio)

test_indices = list(range(test_size))
train_indices = list(range(test_size, total_len))

In [7]:
#OpenAI 포맷으로 변환 함수
def format_conversation(sample):
    return {
        "messages" : [
            {"role" : "system", "content" : sample["system_prompt"]},
            *sample["messages"]
        ]
    }
# 분할 및 변환
train_dataset = [format_conversation(dataset[i]) for i in train_indices]
test_dataset = [format_conversation(dataset[i]) for i in test_indices]

# Dataset 객체로 변환
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [8]:
def to_chatml(data):
    messages = data.get("messages") if isinstance(data, dict) and "messages" in data else data

    parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        parts.append(f"<|im_start|>{role}\n{content}<|im_end|>")
    return "\n".join(parts)

In [9]:
def extract_examples(chatml:str) -> List[Dict[str, str]]:
    examples : List[Dict[str, str]] = []
    pattern = re.compile(r'<\|im_start\|>assistant(.*?)(?=<\|im_end\|>)', re.DOTALL)

    for match in pattern.finditer(chatml):
        start_idx = match.start()
        input_text = chatml[:start_idx].strip() + '\n<|im_start|>assistant'
        label_text = match.group(1).strip()
        examples.append({
            "input" : input_text,
            "label" : label_text
        })

    return examples

In [10]:
prompt_lst = []
label_lst = []

for item in test_dataset:
    chatml = to_chatml(item)
    examples  = extract_examples(chatml)

    for ex in examples:
        prompt_lst.append(ex["input"])
        label_lst.append(ex["label"])

In [11]:
print(label_lst[0])

<tool_call>
{"name": "get_crypto_news", "arguments": {"query": "비트코인", "top_k": 3}}
</tool_call>


#2. 모델 호출

In [12]:
sampling_params = SamplingParams(
    temperature = 0,
    max_tokens = 2048,
    stop = ["<|im_end|>"]
)

In [13]:
llm = LLM(model = "Qwen/Qwen2.5-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 12-28 13:04:14 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 12-28 13:04:14 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

INFO 12-28 13:04:25 [weight_utils.py:281] Time spent downloading weights for Qwen/Qwen2.5-7B-Instruct: 5.862246 seconds


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 12-28 13:04:28 [loader.py:447] Loading weights took 2.18 seconds
INFO 12-28 13:04:28 [gpu_model_runner.py:1186] Model loading took 14.2487 GB and 8.607964 seconds
INFO 12-28 13:04:35 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/0c6741e1d7/rank_0_0 for vLLM's torch.compile
INFO 12-28 13:04:35 [backends.py:425] Dynamo bytecode transform time: 7.52 s
INFO 12-28 13:04:38 [backends.py:132] Cache the graph of shape None for later use
INFO 12-28 13:05:00 [backends.py:144] Compiling a graph for general shape takes 23.42 s
INFO 12-28 13:05:14 [monitor.py:33] torch.compile takes 30.94 s in total
INFO 12-28 13:05:15 [kv_cache_utils.py:566] GPU KV cache size: 951,664 tokens
INFO 12-28 13:05:15 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 29.04x
INFO 12-28 13:05:43 [gpu_model_runner.py:1534] Graph capturing finished in 28 secs, took 0.48 GiB
INFO 12-28 13:05:43 [core.py:151] init engine (profile, create kv cache, warmup model) too

In [15]:
base_model_outputs = llm.generate(prompt_lst, sampling_params)
base_model_text_results = [sample.outputs[0].text.strip() for sample in base_model_outputs]

Processed prompts: 100%|██████████| 585/585 [00:08<00:00, 66.30it/s, est. speed input: 87591.29 toks/s, output: 4879.10 toks/s]  


In [16]:
print(base_model_text_results[10])

<tool_call>
{"name": "search_crypto_term", "arguments": {"query": "샤프 지수", "top_k": 1}}
</tool_call>


# 3. 평가 결과 저장

In [17]:
df = pd.DataFrame({
    "prompt" : prompt_lst,
    "label" : label_lst,
    "output" : base_model_text_results
})

df.to_csv("base_model_evaluation_results.csv", index = False, encoding = "utf-8-sig")

# 4. 평가

In [18]:
# BFCL 확장 평가
import re
import json

def evaluate_function_calls_bfcl(labels, predictions):
    """
    BFCL 스타일 function calling 평가 함수
    """

    results = {
        "tool_selection": {"correct": 0, "total": 0},
        "params_selection": {"correct": 0, "total": 0},
        "params_value_accuracy": {"correct": 0.0, "total": 0},
        "hallucinated_args": {"count": 0},
        "over_call": {"count": 0},
        "under_call": {"count": 0},
        "format_error": {"count": 0}
    }

    tool_call_pattern = re.compile(r"<tool_call>(.*?)</tool_call>", re.DOTALL)
    tool_call_count = 0

    for label, pred in zip(labels, predictions):

        label_match = tool_call_pattern.search(label)
        pred_match = tool_call_pattern.search(pred)

        # ---------- BFCL: over-call / under-call ----------
        if label_match and not pred_match:
            results["under_call"]["count"] += 1
            continue

        if not label_match and pred_match:
            results["over_call"]["count"] += 1
            continue

        if not label_match:
            continue

        tool_call_count += 1

        # ---------- format robustness ----------
        try:
            label_json = json.loads(label_match.group(1))
            pred_json = json.loads(pred_match.group(1))
        except Exception:
            results["format_error"]["count"] += 1
            continue

        # ---------- 1. Tool selection ----------
        results["tool_selection"]["total"] += 1
        if label_json.get("name") == pred_json.get("name"):
            results["tool_selection"]["correct"] += 1

        # ---------- 2. Parameter selection ----------
        label_params = set(label_json.get("arguments", {}).keys())
        pred_params = set(pred_json.get("arguments", {}).keys())

        # 정답 파라미터 기준 평가
        for p in label_params:
            results["params_selection"]["total"] += 1
            if p in pred_params:
                results["params_selection"]["correct"] += 1

        # 환각 파라미터
        hallucinated = pred_params - label_params
        results["hallucinated_args"]["count"] += len(hallucinated)
        results["params_selection"]["total"] += len(hallucinated)

        # ---------- 3. Parameter value accuracy (partial scoring) ----------
        common_params = label_params & pred_params
        if common_params:
            match_cnt = 0
            for k in common_params:
                if label_json["arguments"][k] == pred_json["arguments"][k]:
                    match_cnt += 1

            results["params_value_accuracy"]["correct"] += (
                match_cnt / len(common_params)
            )
            results["params_value_accuracy"]["total"] += 1

    # ---------- 최종 BFCL-style 지표 ----------
    final_results = {
        "tool_accuracy": (
            results["tool_selection"]["correct"] /
            results["tool_selection"]["total"]
            if results["tool_selection"]["total"] > 0 else 0.0
        ),
        "param_key_accuracy": (
            results["params_selection"]["correct"] /
            results["params_selection"]["total"]
            if results["params_selection"]["total"] > 0 else 0.0
        ),
        "param_value_accuracy": (
            results["params_value_accuracy"]["correct"] /
            results["params_value_accuracy"]["total"]
            if results["params_value_accuracy"]["total"] > 0 else 0.0
        ),
        "hallucination_rate": (
            results["hallucinated_args"]["count"] / tool_call_count
            if tool_call_count > 0 else 0.0
        ),
        "over_call_rate": (
            results["over_call"]["count"] / len(labels)
        ),
        "under_call_rate": (
            results["under_call"]["count"] / len(labels)
        ),
        "format_error_rate": (
            results["format_error"]["count"] / tool_call_count
            if tool_call_count > 0 else 0.0
        ),
        "total_tool_calls": tool_call_count
    }

    return final_results


In [19]:
labels = df["label"].tolist()
preds = df["output"].tolist()

results = evaluate_function_calls_bfcl(labels, preds)

for metric, value in results.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2%}")
    else:
        print(f"{metric}: {value}")


tool_accuracy: 93.30%
param_key_accuracy: 83.28%
param_value_accuracy: 52.01%
hallucination_rate: 7.59%
over_call_rate: 0.85%
under_call_rate: 3.08%
format_error_rate: 0.00%
total_tool_calls: 224
